# Demostración del Análisis de Carga Mutacional (TMB)

Este notebook demuestra cómo utilizar el método `calculate_tmb_analysis` de PyMutation para calcular la Carga Mutacional Tumoral (TMB) y generar los archivos de análisis correspondientes.

## ¿Qué es TMB?
La Carga Mutacional Tumoral (TMB) es una medida del número de mutaciones presentes en un tumor, normalizada por el tamaño del genoma interrogado. Es un biomarcador importante en oncología.


## Configuración Inicial


In [1]:
import sys
import os

# Configurar el path para importar pyMut
project_root = os.path.abspath('../../src')
if project_root not in sys.path:
    sys.path.append(project_root)

print('✅ PYTHONPATH configurado para incluir:', project_root)


✅ PYTHONPATH configurado para incluir: /home/luisruimore/Escritorio/TFG/src


In [2]:
# Importar PyMutation y cargar datos
from pyMut.input import read_maf
from pyMut.analysis.mutation_burden import add_tmb_method_to_pymutation

# Asegurar que el método TMB esté disponible
add_tmb_method_to_pymutation()

print('✅ Módulos importados correctamente')


✅ Módulos importados correctamente


## Cargar Datos de Ejemplo


In [3]:
# Cargar el archivo MAF de ejemplo
maf_path = os.path.join(project_root, 'pyMut', 'data', 'examples', 'tcga_laml.maf.gz')

print(f'📂 Cargando archivo: {maf_path}')
py_mutation = read_maf(maf_path)

print(f"✅ Datos cargados exitosamente")
print(f"📊 Forma de los datos: {py_mutation.data.shape}")
print(f"👥 Número de muestras: {len(py_mutation.samples)}")
print(f"🧬 Primeras 3 muestras: {py_mutation.samples[:3]}")


2025-07-10 22:56:42,618 | INFO | pyMut.input | Starting MAF reading: /home/luisruimore/Escritorio/TFG/src/pyMut/data/examples/tcga_laml.maf.gz
2025-07-10 22:56:42,619 | INFO | pyMut.input | Loading from cache: /home/luisruimore/Escritorio/TFG/src/pyMut/data/examples/.pymut_cache/tcga_laml.maf_1d12b7270b4f7707.parquet
2025-07-10 22:56:42,655 | INFO | pyMut.input | Cache loaded successfully in 0.04 seconds


📂 Cargando archivo: /home/luisruimore/Escritorio/TFG/src/pyMut/data/examples/tcga_laml.maf.gz
✅ Datos cargados exitosamente
📊 Forma de los datos: (2207, 216)
👥 Número de muestras: 193
🧬 Primeras 3 muestras: ['TCGA-AB-2988', 'TCGA-AB-2869', 'TCGA-AB-3009']


## Explorar las Columnas de Clasificación de Variantes

Antes de ejecutar el análisis TMB, veamos qué columnas de clasificación de variantes están disponibles:


In [4]:
# Buscar columnas de clasificación de variantes
import re

pattern = re.compile(r'^(gencode_\d+_)?variant[_]?classification$', flags=re.IGNORECASE)
variant_cols = [col for col in py_mutation.data.columns if pattern.match(col)]

print("🔍 Columnas de clasificación de variantes encontradas:")
if variant_cols:
    for i, col in enumerate(variant_cols, 1):
        print(f"  {i}. {col}")
else:
    print("  ❌ No se encontraron columnas de clasificación de variantes")

# Mostrar algunas columnas que contienen 'variant' en el nombre
variant_like_cols = [col for col in py_mutation.data.columns if 'variant' in col.lower()]
print(f"\n🔍 Columnas que contienen 'variant' ({len(variant_like_cols)}):")
for col in variant_like_cols[:5]:  # Mostrar solo las primeras 5
    print(f"  • {col}")


🔍 Columnas de clasificación de variantes encontradas:
  1. Variant_Classification

🔍 Columnas que contienen 'variant' (2):
  • Variant_Classification
  • Variant_Type


## Ejecutar Análisis TMB

Ahora ejecutaremos el análisis de carga mutacional. El método generará dos archivos:
1. **TMB_analysis.tsv**: Análisis por muestra con conteos de mutaciones y TMB normalizado
2. **TMB_statistics.tsv**: Estadísticas globales (media, mediana, cuartiles, etc.)


In [5]:
# Crear directorio para los resultados
output_dir = "results_tmb"
os.makedirs(output_dir, exist_ok=True)

print(f"📁 Directorio de salida: {output_dir}")


📁 Directorio de salida: results_tmb


In [6]:
# Ejecutar el análisis TMB
print("🧬 Ejecutando análisis de carga mutacional...")
print("⏳ Esto puede tomar unos momentos...")

try:
    # Ejecutar análisis TMB con configuración estándar para WES
    results = py_mutation.calculate_tmb_analysis(
        genome_size_bp=60456963,  # Tamaño estándar para WES
        output_dir=output_dir,
        save_files=True
    )
    
    print("✅ Análisis TMB completado exitosamente!")
    
except Exception as e:
    print(f"❌ Error durante el análisis TMB: {e}")
    results = None


2025-07-10 22:56:42,761 | INFO | pyMut.analysis.mutation_burden | Auto-detected variant classification column: Variant_Classification


🧬 Ejecutando análisis de carga mutacional...
⏳ Esto puede tomar unos momentos...


2025-07-10 22:56:58,414 | INFO | pyMut.analysis.mutation_burden | TMB analysis saved to: results_tmb/TMB_analysis.tsv
2025-07-10 22:56:58,414 | INFO | pyMut.analysis.mutation_burden | TMB statistics saved to: results_tmb/TMB_statistics.tsv
2025-07-10 22:56:58,414 | INFO | pyMut.analysis.mutation_burden | Analyzed 193 samples with 2207 total mutations
2025-07-10 22:56:58,415 | INFO | pyMut.analysis.mutation_burden | ========================================
2025-07-10 22:56:58,415 | INFO | pyMut.analysis.mutation_burden | TMB ANALYSIS SUMMARY
2025-07-10 22:56:58,415 | INFO | pyMut.analysis.mutation_burden | ========================================
2025-07-10 22:56:58,415 | INFO | pyMut.analysis.mutation_burden | • Total samples analyzed: 193
2025-07-10 22:56:58,416 | INFO | pyMut.analysis.mutation_burden | • Average total mutations per sample: 11.4
2025-07-10 22:56:58,416 | INFO | pyMut.analysis.mutation_burden | • Average non-synonymous mutations per sample: 9.0
2025-07-10 22:56:58,416 

✅ Análisis TMB completado exitosamente!


## Explorar los Resultados


In [7]:
if results:
    # Obtener los DataFrames de resultados
    analysis_df = results['analysis']
    statistics_df = results['statistics']
    
    print("📊 RESULTADOS DEL ANÁLISIS TMB")
    print("=" * 50)
    print(f"👥 Muestras analizadas: {len(analysis_df)}")
    print(f"📈 Métricas calculadas: {len(statistics_df)}")
    
    # Mostrar las primeras filas del análisis por muestra
    print("\n🔍 Primeras 5 muestras del análisis:")
    print("-" * 40)
    display(analysis_df.head())
    
else:
    print("❌ No se pudieron obtener los resultados del análisis")


📊 RESULTADOS DEL ANÁLISIS TMB
👥 Muestras analizadas: 193
📈 Métricas calculadas: 4

🔍 Primeras 5 muestras del análisis:
----------------------------------------


,Sample,Total_Mutations,Non_Synonymous_Mutations,TMB_Total_Normalized,TMB_Non_Synonymous_Normalized
0,TCGA-AB-2988,15,13,0.248110,0.215029
1,TCGA-AB-2869,12,8,0.198488,0.132326
2,TCGA-AB-3009,42,34,0.694709,0.562384
3,TCGA-AB-2830,17,13,0.281192,0.215029
4,TCGA-AB-2887,15,12,0.248110,0.198488


## Estadísticas Globales


In [8]:
if results:
    print("📈 ESTADÍSTICAS GLOBALES DEL TMB")
    print("=" * 40)
    display(statistics_df)
    
    # Mostrar algunas estadísticas clave
    print("\n🎯 ESTADÍSTICAS CLAVE:")
    print("-" * 30)
    
    # TMB total normalizado
    tmb_total_stats = statistics_df[statistics_df['Metric'] == 'TMB_Total_Normalized'].iloc[0]
    print(f"🧬 TMB Total Normalizado:")
    print(f"   • Media: {tmb_total_stats['Mean']:.4f} mutaciones/Mb")
    print(f"   • Mediana: {tmb_total_stats['Median']:.4f} mutaciones/Mb")
    print(f"   • Rango: {tmb_total_stats['Min']:.4f} - {tmb_total_stats['Max']:.4f} mutaciones/Mb")
    
    # TMB no sinónimo normalizado
    tmb_nonsyn_stats = statistics_df[statistics_df['Metric'] == 'TMB_Non_Synonymous_Normalized'].iloc[0]
    print(f"\n🎯 TMB No Sinónimo Normalizado:")
    print(f"   • Media: {tmb_nonsyn_stats['Mean']:.4f} mutaciones/Mb")
    print(f"   • Mediana: {tmb_nonsyn_stats['Median']:.4f} mutaciones/Mb")
    print(f"   • Rango: {tmb_nonsyn_stats['Min']:.4f} - {tmb_nonsyn_stats['Max']:.4f} mutaciones/Mb")


📈 ESTADÍSTICAS GLOBALES DEL TMB


,Metric,Count,Mean,Median,Min,Max,Q1,Q3,Std
0,Total_Mutations,193,11.435233,11.000000,1.000000,42.000000,6.000000,15.000000,6.752870
1,Non_Synonymous_Mutations,193,8.974093,9.000000,0.000000,34.000000,5.000000,12.000000,5.452862
2,TMB_Total_Normalized,193,0.189147,0.181948,0.016541,0.694709,0.099244,0.248110,0.111697
3,TMB_Non_Synonymous_Normalized,193,0.148438,0.148866,0.000000,0.562384,0.082703,0.198488,0.090194



🎯 ESTADÍSTICAS CLAVE:
------------------------------
🧬 TMB Total Normalizado:
   • Media: 0.1891 mutaciones/Mb
   • Mediana: 0.1819 mutaciones/Mb
   • Rango: 0.0165 - 0.6947 mutaciones/Mb

🎯 TMB No Sinónimo Normalizado:
   • Media: 0.1484 mutaciones/Mb
   • Mediana: 0.1489 mutaciones/Mb
   • Rango: 0.0000 - 0.5624 mutaciones/Mb
